In [1]:
import pandas as pd
import re

# Load Sam's proteomics files

In [2]:
sams_prots = pd.read_excel("/home/gus/MEGAsync/zim/main/SamsProteomics/peptide_data/pr151026_ms151023_SR_protein.xlsx",
                           sheetname="Sheet1",
                           skiprows=[0,1]
                          )

In [3]:
sams_prots.head()

,Accession,Peptide count,Unique peptides,Confidence score,Anova (p),Max fold change,Highest mean condition,Lowest mean condition,Description,151023_o2_02_in151022_an_SR1-pulse,...,151023_o2_04_in151022_an_SR3-pulse,151023_o2_05_in151022_an_SR4-pulse,151023_o2_02_in151022_an_SR1-pulse.1,151023_o2_03_in151022_an_SR2-pulse.1,151023_o2_04_in151022_an_SR3-pulse.1,151023_o2_05_in151022_an_SR4-pulse.1,151023_o2_02_in151022_an_SR1-pulse.2,151023_o2_03_in151022_an_SR2-pulse.2,151023_o2_04_in151022_an_SR3-pulse.2,151023_o2_05_in151022_an_SR4-pulse.2
0,ASTEI05705-PA,274,271,18696.33,NaN,1.0728,SR2,SR3,hypothetical protein|protein_coding|scaffold_...,1.004464e+10,...,9.888590e+09,1.022448e+10,8.163486e+09,1.060844e+10,9.425724e+09,1.025766e+10,283,299,263,279
1,ASTEI08310-PA,194,190,18318.32,NaN,1.22286,SR1,SR3,hypothetical protein|protein_coding|scaffold_...,1.368064e+11,...,1.118741e+11,1.291522e+11,1.111853e+11,1.311113e+11,1.066375e+11,1.295713e+11,267,276,274,276
2,ASTEI01466-PA,168,164,11120.27,NaN,1.06332,SR2,SR3,hypothetical protein|protein_coding|scaffold_...,5.035113e+09,...,4.744827e+09,4.890575e+09,4.092138e+09,5.045285e+09,4.522731e+09,4.906444e+09,153,150,150,150
3,ASTEI02092-PA;ASTEI04446-PA,119,114,7116.22,NaN,1.0765,SR2,SR3,hypothetical protein|protein_coding|scaffold_...,4.649045e+09,...,4.385391e+09,4.486687e+09,3.778373e+09,4.720867e+09,4.180119e+09,4.501246e+09,104,108,106,102
4,ASTEI00150-PA,104,102,7110.97,NaN,1.14679,SR1,SR2,hypothetical protein|protein_coding|scaffold_...,3.163588e+09,...,2.980585e+09,2.956743e+09,2.571112e+09,2.758636e+09,2.841070e+09,2.966337e+09,101,93,96,104


In [4]:
prot_set = set()

for row in list(sams_prots.Accession.apply(lambda r: r.split(';'))):
    prot_set.update(row)

prot_list = [x[:-3] for x in list(prot_set)]  

In [5]:
prot_list[:6]

[u'ASTEI11137',
 u'ASTEI02810',
 u'ASTEI00393',
 u'ASTEI07414',
 u'ASTEI04291',
 u'ASTEI01662']

In [6]:
'ASTEI097191' in prot_list

True

In [7]:
sams_data = pd.DataFrame(data=prot_list,columns=['TARGET_PROTS']
                         
                        )
sams_data.head()

,TARGET_PROTS
0,ASTEI11137
1,ASTEI02810
2,ASTEI00393
3,ASTEI07414
4,ASTEI04291


# Load conversion table from ASTEI to ASTE

In [8]:
def split_and_fold(df,col_name,pattern):
    df = df.copy()
    s = df[col_name].str.split(pattern).apply(pd.Series, 1).stack()
    s.index = s.index.droplevel(-1) # to line up with df's index
    s.name = col_name # needs a name to join
    del df[col_name]
    return df.join(s)

In [9]:
mapping = pd.read_csv("/home/gus/MEGAsync/zim/main/SamsProteomics/peptide_data/Anopheles_stephensi_MAPPINGS_AsteS1.2_AsteI2.2.csv", sep='\t')

In [10]:
mapping.head()

,mapping_type,overlap,exon_match,ids_1,length_1,overlap_1,ids_2,length_2,overlap_2
0,many_to_one,1131,2,"ASTE000001-RA,ASTE000002-RA",1197,0.9449,ASTEI07101-RA,1131,1
1,one_to_one,2553,5,ASTE000003-RA,2553,1.0000,ASTEI07102-RA,2553,1
2,one_to_one,453,0,ASTE000004-RA,564,0.8032,ASTEI07103-RA,453,1
3,one_to_one,669,0,ASTE000005-RA,861,0.7770,ASTEI07104-RA,669,1
4,one_to_one,2886,1,ASTE000006-RA,2886,1.0000,ASTEI07105-RA,2886,1


In [11]:
mapping = mapping[['mapping_type','ids_1','ids_2']]

In [12]:
mapping.head()

,mapping_type,ids_1,ids_2
0,many_to_one,"ASTE000001-RA,ASTE000002-RA",ASTEI07101-RA
1,one_to_one,ASTE000003-RA,ASTEI07102-RA
2,one_to_one,ASTE000004-RA,ASTEI07103-RA
3,one_to_one,ASTE000005-RA,ASTEI07104-RA
4,one_to_one,ASTE000006-RA,ASTEI07105-RA


In [13]:
mapping_folded = split_and_fold(df=split_and_fold(df=mapping,col_name='ids_1',pattern=','),col_name='ids_2',pattern=',')

In [14]:
mapping_folded.drop_duplicates(inplace=True)

In [15]:
mapping_folded.head()

,mapping_type,ids_1,ids_2
0,many_to_one,ASTE000001-RA,ASTEI07101-RA
0,many_to_one,ASTE000002-RA,ASTEI07101-RA
1,one_to_one,ASTE000003-RA,ASTEI07102-RA
2,one_to_one,ASTE000004-RA,ASTEI07103-RA
3,one_to_one,ASTE000005-RA,ASTEI07104-RA


In [16]:
mapping_folded['ids_1'] = mapping_folded.ids_1.apply(lambda x: x[:-3])
mapping_folded['ids_2'] = mapping_folded.ids_2.apply(lambda x: x[:-3])

In [17]:
mapping_folded.head()

,mapping_type,ids_1,ids_2
0,many_to_one,ASTE000001,ASTEI07101
0,many_to_one,ASTE000002,ASTEI07101
1,one_to_one,ASTE000003,ASTEI07102
2,one_to_one,ASTE000004,ASTEI07103
3,one_to_one,ASTE000005,ASTEI07104


# Get TARGET_PROTS as ASTE gene names

In [18]:
target_aste = pd.merge(left=mapping_folded, right=sams_data, how='inner',
                       on=None, left_on="ids_2", right_on="TARGET_PROTS"
                      )

In [19]:
target_aste.head()

,mapping_type,ids_1,ids_2,TARGET_PROTS
0,one_to_many,ASTE000013,ASTEI08291,ASTEI08291
1,one_to_one,ASTE000014,ASTEI11137,ASTEI11137
2,one_to_one,ASTE000015,ASTEI11136,ASTEI11136
3,one_to_one,ASTE000016,ASTEI08290,ASTEI08290
4,one_to_one,ASTE000053,ASTEI01972,ASTEI01972


In [20]:
target_aste = target_aste.drop('TARGET_PROTS', axis=1)

In [21]:
target_aste.head()

,mapping_type,ids_1,ids_2
0,one_to_many,ASTE000013,ASTEI08291
1,one_to_one,ASTE000014,ASTEI11137
2,one_to_one,ASTE000015,ASTEI11136
3,one_to_one,ASTE000016,ASTEI08290
4,one_to_one,ASTE000053,ASTEI01972


In [22]:
target_aste.rename(columns={"mapping_type":"mapping_type_aste_to_astei","ids_1":"aste_target","ids_2":"astei_target"},inplace=True)

In [23]:
target_aste.head()

,mapping_type_aste_to_astei,aste_target,astei_target
0,one_to_many,ASTE000013,ASTEI08291
1,one_to_one,ASTE000014,ASTEI11137
2,one_to_one,ASTE000015,ASTEI11136
3,one_to_one,ASTE000016,ASTEI08290
4,one_to_one,ASTE000053,ASTEI01972


In [24]:
print len(sams_data)
print len(target_aste)

1255
1371


# Load OrthoDB8 ~~Dm_As_Ag~~ Diptera ortholog table

http://orthodb.org/

### Page options used:

- Selected ~~_Anopheles gambiae_, _An stephensi_, and _Drosophila melanogaster_~~ Diptera
- No filters as to type of orthologs returned (single copy etc)
- No search terms supplied
- clicked submit
- clicked download as table

Use this link to replicate AgAsDm search: http://tiny.cc/ortho8_AgAsDm

Use this link to replicate Diptera search: http://tiny.cc/ortho8_Diptera_rund

In [25]:
# AGASDM
ag_as_dm_path = "/home/gus/MEGAsync/zim/main/SamsProteomics/peptide_data/orthodb8/selected_species_nodes_tab_data/orthodb8_Dm_As_Ag_tab_data.csv"

# all levels dump from orthoDB8
odb8_all_path = "/home/gus/MEGAsync/zim/main/SamsProteomics/peptide_data/orthodb8/ODB8_EukOGs_genes_ALL_levels.txt"

# diptera
diptera_path = "/home/gus/MEGAsync/zim/main/SamsProteomics/peptide_data/orthodb8/ODB8_EukOGs_genes_ALL_levels.grep_7147:Diptera.txt"

### First pull out only dipteran data from the `all_levels` file

In [26]:
!head -n 1 $odb8_all_path > $diptera_path
!grep '7147:Diptera' $odb8_all_path >> $diptera_path

In [27]:
!head $diptera_path

odb8_level	odb8_og_id	protein_id	odb8_prot_id	organism	uniprot_description
7147:Diptera	EOG808R20	AALB007334-PA	7167:001ca5	Anopheles albimanus	
7147:Diptera	EOG80CM7H	5512162.p	7370:0017f8	Musca domestica	
7147:Diptera	EOG80CM7K	3231350.p	7370:0020f5	Musca domestica	
7147:Diptera	EOG80GGGX	AMEM002228-PA	30066:00068f	Anopheles merus	
7147:Diptera	EOG80GGHT	3451754.p	7370:004304	Musca domestica	
7147:Diptera	EOG80GGJ3	5507785.p	7370:000228	Musca domestica	
7147:Diptera	EOG80VZDR	AMEC015278-PA	34690:002a9b	Anopheles melas	
7147:Diptera	EOG80VZF7	CPIJ000359-PA	7176:000152	Culex quinquefasciatus	Nop2p
7147:Diptera	EOG80VZF9	3079430.p	7370:002762	Musca domestica	


### Continue with only dipteran file after mild cleaning

- drop any record where 
    ```python
    isinstance(protein_id, NaN) == True
    ```

In [28]:
diptera = pd.read_csv(diptera_path, sep='\t')

In [29]:
diptera.head()

,odb8_level,odb8_og_id,protein_id,odb8_prot_id,organism,uniprot_description
0,7147:Diptera,EOG808R20,AALB007334-PA,7167:001ca5,Anopheles albimanus,NaN
1,7147:Diptera,EOG80CM7H,5512162.p,7370:0017f8,Musca domestica,NaN
2,7147:Diptera,EOG80CM7K,3231350.p,7370:0020f5,Musca domestica,NaN
3,7147:Diptera,EOG80GGGX,AMEM002228-PA,30066:00068f,Anopheles merus,NaN
4,7147:Diptera,EOG80GGHT,3451754.p,7370:004304,Musca domestica,NaN


In [30]:
diptera = diptera.dropna(subset=['protein_id'])

In [31]:
ensembl_product_suffix = re.compile(r'-[PR].\b')

In [32]:
def test_prot_id(x):
    try:
        found = ensembl_product_suffix.search(x)
        if found is None:
            return False
        else:
            return True
    except TypeError:
        print x

def ensembl_prot_to_gene(df):
    df = df.copy()
    df['ensembl_gene_id'] = df.protein_id.apply(lambda x: x[:-3] if test_prot_id(x) else x)
    
    return df

In [33]:
diptera = ensembl_prot_to_gene(diptera)

# Get `pub_og_ids` that contain the ASTE target genes

In [34]:
target_aste_ortho_groups = pd.merge(left=diptera, right=target_aste, how='inner',
                                    left_on="ensembl_gene_id", right_on="aste_target"
                                   )

In [35]:
len(target_aste_ortho_groups)

1346

In [36]:
target_aste_ortho_groups.head()

,odb8_level,odb8_og_id,protein_id,odb8_prot_id,organism,uniprot_description,ensembl_gene_id,mapping_type_aste_to_astei,aste_target,astei_target
0,7147:Diptera,EOG80048M,ASTE003050-PA,30069:000be9,Anopheles stephensi,NaN,ASTE003050,one_to_one,ASTE003050,ASTEI05206
1,7147:Diptera,EOG80048S,ASTE011524-PA,30069:002d03,Anopheles stephensi,NaN,ASTE011524,one_to_one,ASTE011524,ASTEI02453
2,7147:Diptera,EOG80048V,ASTE005096-PA,30069:0013e7,Anopheles stephensi,NaN,ASTE005096,one_to_one,ASTE005096,ASTEI10108
3,7147:Diptera,EOG80048W,ASTE005418-PA,30069:001529,Anopheles stephensi,NaN,ASTE005418,one_to_one,ASTE005418,ASTEI01149
4,7147:Diptera,EOG800494,ASTE011590-PA,30069:002d45,Anopheles stephensi,NaN,ASTE011590,one_to_one,ASTE011590,ASTEI06289


In [37]:
target_og_ids = pd.DataFrame(target_aste_ortho_groups.odb8_og_id)

In [38]:
target_og_ids.head()

,odb8_og_id
0,EOG80048M
1,EOG80048S
2,EOG80048V
3,EOG80048W
4,EOG800494


# Use the `pub_og_ids` from above to pull out all member genes from any dipteran

In [39]:
target_diptera = pd.merge(left=diptera, right=target_og_ids, how='inner',
                          left_on="odb8_og_id", right_on="odb8_og_id"
                         )

In [40]:
len(target_diptera.query("""organism == 'Anopheles gambiae'"""))

1865

In [41]:
target_diptera.head()

,odb8_level,odb8_og_id,protein_id,odb8_prot_id,organism,uniprot_description,ensembl_gene_id
0,7147:Diptera,EOG80CM7K,3231350.p,7370:0020f5,Musca domestica,NaN,3231350.p
1,7147:Diptera,EOG80CM7K,AAEL010901-PA,7159:0028d4,Aedes aegypti,AAEL010901-PA,AAEL010901
2,7147:Diptera,EOG80CM7K,AALB004306-PA,7167:0010d1,Anopheles albimanus,NaN,AALB004306
3,7147:Diptera,EOG80CM7K,AARA002395-PA,7173:00095a,Anopheles arabiensis,NaN,AARA002395
4,7147:Diptera,EOG80CM7K,AATE004704-PA,41427:000bee,Anopheles atroparvus,NaN,AATE004704


# Build results table and write to file

In [42]:
target_aste_ortho_groups.head()

,odb8_level,odb8_og_id,protein_id,odb8_prot_id,organism,uniprot_description,ensembl_gene_id,mapping_type_aste_to_astei,aste_target,astei_target
0,7147:Diptera,EOG80048M,ASTE003050-PA,30069:000be9,Anopheles stephensi,NaN,ASTE003050,one_to_one,ASTE003050,ASTEI05206
1,7147:Diptera,EOG80048S,ASTE011524-PA,30069:002d03,Anopheles stephensi,NaN,ASTE011524,one_to_one,ASTE011524,ASTEI02453
2,7147:Diptera,EOG80048V,ASTE005096-PA,30069:0013e7,Anopheles stephensi,NaN,ASTE005096,one_to_one,ASTE005096,ASTEI10108
3,7147:Diptera,EOG80048W,ASTE005418-PA,30069:001529,Anopheles stephensi,NaN,ASTE005418,one_to_one,ASTE005418,ASTEI01149
4,7147:Diptera,EOG800494,ASTE011590-PA,30069:002d45,Anopheles stephensi,NaN,ASTE011590,one_to_one,ASTE011590,ASTEI06289


In [43]:
target_aste_ortho_groups[["odb8_og_id","mapping_type_aste_to_astei","aste_target","astei_target"]].head()

,odb8_og_id,mapping_type_aste_to_astei,aste_target,astei_target
0,EOG80048M,one_to_one,ASTE003050,ASTEI05206
1,EOG80048S,one_to_one,ASTE011524,ASTEI02453
2,EOG80048V,one_to_one,ASTE005096,ASTEI10108
3,EOG80048W,one_to_one,ASTE005418,ASTEI01149
4,EOG800494,one_to_one,ASTE011590,ASTEI06289


In [44]:
# join the original ASTEI gene names to make it easier to use
selected_columns = ["odb8_og_id","mapping_type_aste_to_astei","aste_target","astei_target"]
target_diptera_joined_ASTEI = pd.merge(left=target_diptera,
                                       right=target_aste_ortho_groups[selected_columns],
                                       how='inner', on="odb8_og_id"
                                      )
target_diptera_joined_ASTEI.head()

,odb8_level,odb8_og_id,protein_id,odb8_prot_id,organism,uniprot_description,ensembl_gene_id,mapping_type_aste_to_astei,aste_target,astei_target
0,7147:Diptera,EOG80CM7K,3231350.p,7370:0020f5,Musca domestica,NaN,3231350.p,one_to_one,ASTE000566,ASTEI10787
1,7147:Diptera,EOG80CM7K,AAEL010901-PA,7159:0028d4,Aedes aegypti,AAEL010901-PA,AAEL010901,one_to_one,ASTE000566,ASTEI10787
2,7147:Diptera,EOG80CM7K,AALB004306-PA,7167:0010d1,Anopheles albimanus,NaN,AALB004306,one_to_one,ASTE000566,ASTEI10787
3,7147:Diptera,EOG80CM7K,AARA002395-PA,7173:00095a,Anopheles arabiensis,NaN,AARA002395,one_to_one,ASTE000566,ASTEI10787
4,7147:Diptera,EOG80CM7K,AATE004704-PA,41427:000bee,Anopheles atroparvus,NaN,AATE004704,one_to_one,ASTE000566,ASTEI10787


In [47]:
selected_columns = ['odb8_og_id','protein_id','odb8_prot_id','organism','uniprot_description','ensembl_gene_id','mapping_type_aste_to_astei','aste_target','astei_target']
target_diptera_joined_ASTEI.to_csv("/home/gus/MEGAsync/zim/main/SamsProteomics/peptide_data/pr151026_ms151023_SR_protein.orthodb8_orthogroups_diptera.csv",
                                     index=False,
                                     columns=selected_columns
                                    )